# Reading Activity 27 - Sampling Methods

## Objectives

+ Define the basics of Markov Chains (transition kernels, invariance, ergodicity, detailed balance condition).
+ Define Mark

## References

+ These notes.
+ Chapter 11 of Bishop 2006.

## Problem Definition

We have seen that the Bayesian formulation of inverse problems results in intractable posterior distributions.
In particular, these posteriors are known only up to a normalization constant.
In the next series of lectures, we are going to develop methodologies that allows us to sample from these distributions.
The most celebrated of these methodologies is Markov Chain Monte Carlo (MCMC) which has at its core the Metropolis algorithms.
It is a long way to go, but we can state right away the problem definition and what we are set out to do.

Without loss of generality, let $X\in\mathcal{X}\subset\mathbb{R}^d$ be random variable with an arbitrary probability density, say $\pi(x)$ known up to a normalization constant.
That is, we have that:
$$
\pi(x) = \frac{h(x)}{Z},
$$
where $h(x)$ is a known function that we can evaluate at will, but $Z$ is not known.
Our goal is to generate samples from $\pi(x)$, by only evaluating $h(x)$.
The revolutionary idea of Metropolis was to construct a stochastic process using only $h(x)$ samples from which resemble (in some way we will specify below) samples from $\pi(x)$.
To understand the details, we will have to introduce some key concepts.

## Markov Chains

### Definition
Let $X_n, n=1,2,\dots$ be a stochastic process taking values in $\mathcal{X}\subset\mathbb{R}^d$ whcih could either be discrete or continuous.
We will refer to $n$ as the *time step*.
We say that this stochastic process is a *Markov chain* if the evolution of $X_{n+1}$ depends only on the value of $X_n$ and not on all the history of the process.
Let us define this a little bit more rigorously.

Let $x_1,\dots,x_n\in\mathcal{X}$ be the observed values of the process up to $n$-th time step.
The Markov property can now be expressed as:
$$
p(X_{n+1}=x_{n+1}|X_1=x_1,\dots,X_n=x_n) = p(X_{n+1}=x_{n+1}|X_n=x_n).
$$
If there is no ambiguity, we will be simplifying the notation by dropping the capital letters.
That is, we will be writting:
$$
p(x_{n+1}|x_1,\dots,x_n) = p(x_{n+1}|x_n).
$$
To simplify things even further, we will also use the collective notation:
$$
x_{1:n} = (x_1,\dots,x_n)\in\mathcal{X}^n.
$$
With this notation, we can re-write the Markov property in even simpler terms:
$$
p(x_{n+1}|x_{1:n}) = p(x_{n+1}|x_n).
$$

### The joint distribution of a Markov chain
The *joint distribution* is defined as:
$$
p(x_{1:n}) := P(X_1=x_1,\dots,X_n=x_n).
$$
If $X_n$ is a Markov chain, then we simply have:
$$
p(x_{1:n}) = p(x_1)p(x_2|x_1)\dots p(x_n|x_{n-1}),
$$
or
$$
p(x_{1:n}) = p(x_1)\prod_{t=2}^np(x_t|x_{t-1}).
$$
So, we see that we know the joint distribution of a Markov chain if we know the probability of hoping from one state to the next.
This propability is known as the transition kernel of the Markov chain.

### Transition Kernel
To describe a Markov chain, we only need to know the *transition kernel*.
The transition kernel gives the probability of moving from one state to any other at a given step.
Mathematically, the transition kernel of the $n$-th step is the function:
$$
T_n:\mathcal{X}\times \mathcal{X}\rightarrow \mathbb{R}_+,
$$
defined by:
$$
T_n(x_n, x_{n+1}) = P(X_{n+1}=x_{n+1}|X_n=x_n).
$$
In words, $T_n(x_n, x_{n+1})$ is the probability that at step $n$ we jump from state $x_n$ to to state $x_{n+1}$.

Please, note that the transition kernel, in general, depends on the time step $n$.
We say that the Markov chain is *stationary*, if the transition kernel does not depend on $n$, i.e., if
$$
T_n(x_n, x_{n+1}) = T(x_n,x_{n+1}).
$$
**From now on, we will only consider stationary Markov chains.**
For stationary Markov chains, and when there is now ambiguity, we will be writing:
$$
T(x_n,x_{n+1}) = p(x_{n+1}|x_n).
$$

## Invariant Distributions

Let $X_1,X_2,\dots$ be a Markov chain with transition kernel $T(x,x')$ and $\pi(x)$ be a probability density.
We say that the Markov chain leaves $\pi(x)$ *invariant* if:
$$
\pi(x) = \int \pi(x')T(x',x)dx'.
$$
In words, $\pi(x)$ is invariant if when you start from a sample from it and you follow the Markov chain you get a sample from it.

Invariance is one of the key requirements of a working MCMC algorithm.
Whatever you do, the chain you construct must be invariant with respect to the distribution from which you want to sample.
If you manage to do that, then once you get one sample from your distribution, you can get as many as you want by simply following the transition kernel.



## The Detailed Balance Condition
Checking invariance is not trivial for a generic Markov chain.
However, there is a sufficient condition that guarantees invariance.
This condition is known as the *detailed balance condition* and it is:
$$
\pi(x)T(x,x') = \pi(x')T(x',x).
$$
In words, a Markov chain that satisfies the detailed balance condition is *reversible* in the following sense.
The probability of sampling an $x$ and transitioning to $x'$ is the same as the probability of doing the reverse.

If the detailed balance condition is satisfied, then $\pi(x)$ is an invariant distribution:
$$
\int \pi(x')T(x',x)dx' = \int \pi(x)T(x,x')dx' = \pi(x)\int T(x,x')dx' = \pi(x),
$$
since
$$
\int T(x,x') dx' = \int p(x'|x)dx' = 1.
$$
The reverse does not necessarily hold.
The key idea of Metropolis was to construct a Markov chain that satisfies the detailed balance condition for the distribution you are interested in.

## Ergodicity

A Markov chain may have no invariant distribution (e.g., the random walk does not have an invariant distribution), one, or more than one.
To guarantee uniqueness of the invariant distribution, need *ergodicity*.
To define ergodicity precicely for contiuous Markov chains, we need a little bit of notation.
In words, ergodicity requires that the random variable $X_n$ converges in distribution to $\pi(x)$ irrespective of the starting point.
Obviously, this is not easy to show for a generic Markov chain.
Fortunately, we know that a Markov chain is ergodic if:

+ it is *aperiodic* (i.e., it does not return to the same state at fixed intervals)
+ it is *positive recurrent* (i.e., the expected number of steps for returning to the same state is finite).

## Equilibrium Distribution
If a Markov chain is ergodic and it has an invariant distribution, then that invariant distribution is unique and it is called the *equilibrium distribution*.
The Metropolis algorithms constructs a Markov chain that has a desired equilibrium distribution.

## The Metropolis Algorithm

Now, let's get back to the initial problem of sampling from:
$$
\pi(x) = \frac{h(x)}{Z},
$$
without knowing $Z$.
In 1953, Metropolis et al. demonstrated how we can construct a Markov chain with $\pi(x)$ as the equilibrium density.
The algorithm is based on biasing an underlying symmetric, stationary Markov chain.
Let $T(x,x')$ be the transition kernel of this underlying Markov chain (also called the *proposal distribution*.
The transition kernel must be symmetric, i.e.,
$$
T(x,x') = T(x',x).
$$
A very common choice of the proposal distribution is the random walk transition kernel:
$$
T(x,x') = \mathcal{N}(x'|x, \Sigma).
$$
However, this is just one possibility.
Once we have pick a proposal, we construct the desired Markov chain as follows:

+ **Initialization:** Pick an arbitrary starting point $x_0$.
+ For each time step $n$:
    - **Generation:** Sample a candidate $x$ from $T(x_n, x)$.
    - **Calculation:** Calculate the *acceptance ratio*:
    $$
    \alpha(x_n, x) = \min\left\{1, \frac{h(x)}{h(x_n)}\right\}.
    $$
    This is the only place where you may need to evaluate the underlying model.
    - **Accept/Reject:**
        - Generate a uniform number $u\sim \mathcal{U}([0,1])$.
        - If $u\le \alpha$, *accept* and set $x_{n+1}=x$.
        - If $u > \alpha$, *reject* ad set $x_{n+1} = x_n$.

## Why Does Metropolis Work?

Well, it works because it gives us a Markov chain with the desired equilibrium distribution.
That is, a chain that has an invariant distribution of our choice that it is also ergodic.

To show that $\pi(x)$ is the invariant distribution of the Metropolis Markov chain, we will show that the latter satisfies the detailed balance condition.
To this end, we need the transition kernel of the chain.
The transition kernel $K(x,x')$ gives the probability that the Metropolis chain moves from $x$ to $x'$.
It is:
$$
K(x,x') = T(x,x')\alpha(x,x') + (1 - r(x))\delta(x' - x),
$$
where $T(x,x')$ is the transition kernel of the proposal distribution,
$$
\alpha(x,x') = \min\left\{1, \frac{h(x')}{h(x)}\right\}
$$
is the acceptance ratio,
$$
r(x) = \int T(x, y)\alpha(x, y)dy,
$$
is the probability of accepting any move, i.e., $1 - r(x)$ is the probability of not accepting the move, and $\delta(x-x')$ is the Dirac delta centered at $x'$.

Let's prove that the detailed balance holds for this transition kernel.
For $x = x'$ the equation holds trivially (even though we would have to interpret it slightly differently to be 100% rigorous).
For $x\not= x'$, we have:
$$
\begin{array}{ccc}
\pi(x) K(x, x') &=& \frac{h(x)}{Z} T(x,x')\alpha(x,x') \\
&=& \frac{h(x)}{Z}T(x,x')\min\left\{1, \frac{h(x')}{h(x)}\right\}\\
&=& \frac{h(x')}{h(x')}\frac{h(x)}{Z}T(x,x')\min\left\{1, \frac{h(x')}{h(x)}\right\}\\
&=& \frac{h(x')}{Z}T(x,x')\min\left\{\frac{h(x)}{h(x')},\frac{h(x)}{h(x')}\cdot\frac{h(x')}{h(x)}\right\}\\
&=& \pi(x')T(x,x')\min\left\{\frac{h(x)}{h(x')},1\right\}\\
&=& \pi(x')T(x,x')\alpha(x',x)\\
&=& \pi(x')T(x', x)\alpha(x',x)\\
&=& \pi(x')K(x',x),
\end{array}
$$
where we also made use of the symmetry of the proposl $T(x,x') = T(x',x)$.